In [ ]:
! pip install pyTelegramBotAPI

In [ ]:
import telebot
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("/content/drive/MyDrive/Datasets/Sheykh Recognizer.h5")

mybot = telebot.TeleBot("Token",parse_mode="None")

@mybot.message_handler(commands=['start'])
def send_welcome(message: str):
    user_first_name = str(message.chat.first_name)
    mybot.reply_to(message, 'Hi ' + user_first_name + " welcome to my bot :) ")
    time_to_send = mybot.send_message(message.chat.id,"Send me an image I'll tell you is that person a Sheykh or a normal person")
    mybot.register_next_step_handler(time_to_send, recieve_file)

def recieve_file(message):
  try :
    fileID = message.photo[-1].file_id
    path = fileID+".jpg"
    file_info = mybot.get_file(fileID)
    downloaded_file = mybot.download_file(file_info.file_path)

    with open(path,'wb') as new_file:
      new_file.write(downloaded_file)
    
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = img/255
    img = img.reshape(1, 224,224, 3)
    pred = model.predict(img)
    result = np.argmax(pred)
    
    if result == 0:
      mybot.reply_to(message, 'She/He seems like a normal person')
    elif result == 1:
      mybot.reply_to(message, 'He seems like a Sheykh')

  except:
    mybot.reply_to(message, "Please send a photo\n Try again /start")

@mybot.message_handler(commands=['help'])
def get_img(message):
    mybot.reply_to(message,"/start will recieve an image and detect it's category")

@mybot.message_handler(func = lambda message:True)
def user_sent_anything(message):
    mybot.reply_to(message,"I didn't get what you want")

mybot.polling()